In [26]:
# Importing libraries
import requests
import pandas as pd
import json
from google.colab import files

# Making empty dataframe, df
df = pd.DataFrame(columns=['Year',0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Make call and get dataframe for March of each year, concatinate with df
for i in range(2023,2024):
  call = f"https://api.census.gov/data/{i}/cps/basic/mar?get=GTCBSA,PEHRUSLT,PEDIPGED,PECYC,PEAFEVER,PEAFNOW,PEERNHRO,PEERNLAB,HEFAMINC,HRNUMHOU"
  year = requests.get(call).text
  year = json.loads(year)
  dfyear = pd.DataFrame(year[1:])
  dfyear.insert(0, 'Year', f'{i}')
  df = pd.concat([df, dfyear])

# Rename columns to variable names and change variable data types
df = df.rename(columns={0: "GTCBSA", 1: "PEHRUSLT", 2: "PEDIPGED", 3: "PECYC", 4: "PEAFEVER", 5: "PEAFNOW", 6: "Hours Usually Worked", 7: "PEERNLAB", 8: "Household Income", 9: "Household Members"})
df = df.astype({'Year': int, 'PEHRUSLT': float, "PEDIPGED" :float, "PECYC" :float, "PEAFEVER" :float, "PEAFNOW":float, "Hours Usually Worked":float, "PEERNLAB":float, "Household Income":float, "Household Members":float}).dropna()

# Making recoding dictionaries
PEDIPGED_dict = {-1: 0, 1: 1, 2: 1}
PECYC_dict = {-1: 0, 1:0, 2:1, 3:2, 4:3, 5:4}
PEAFEVER_dict = {2: 0, 1: 1}
PEAFNOW_dict = {2: 0, 1: 1}
PEERNLAB_dict = {-1: 0, 2: 0, 1: 1}

# Creating recoded variables
df['HS or GED'] = df['PEDIPGED'].map(PEDIPGED_dict)
df['Years of College'] = df['PECYC'].map(PECYC_dict)
df['Ever in Military'] = df['PEAFEVER'].map(PEAFEVER_dict)
df['Currently in Military'] = df['PEAFNOW'].map(PEAFNOW_dict)
df['Union Member'] = df['PEERNLAB'].map(PEERNLAB_dict)

df['Year'] = df['Year'] - 2000 # Changing to 2 digit years for st.chart scaling (and to hopefully recreate the Y2K panic in 976 years)

# Taking mean for each city each year
df = df.groupby(['Year', 'GTCBSA']).mean()

# Resetting index
df = df.reset_index()

# Dictionary of GTCBSA and city names taken from census documentation
GTCBSA_dict = {
      "33660": "Mobile, AL",
      "17420": "Cleveland, TN",
      "27100": "Jackson, MI",
      "33460": "Minneapolis-St Paul-Bloomington, MN-WI",
      "33340": "Milwaukee-Waukesha-West Allis, WI",
      "26820": "Idaho Falls, ID",
      "12540": "Bakersfield, CA",
      "12420": "Austin-Round Rock, TX",
      "19820": "Detroit-Warren-Dearborn, MI",
      "29740": "Las Cruces, NM",
      "14260": "Boise City, ID",
      "39300": "Providence-Warwick, RI-MA",
      "31700": "Manchester-Nashua, NH",
      "41180": "St. Louis, MO-IL",
      "17980": "Columbus, GA-AL",
      "32580": "McAllen-Edinburg-Mission, TX",
      "13820": "Birmingham-Hoover, AL",
      "37460": "Panama City, FL",
      "37340": "Palm Bay-Melbourne-Titusville, FL",
      "19340": "Davenport-Moline-Rock Island, IA-IL",
      "24020": "Glen Falls, NY",
      "49660": "Youngstown-Warren-Boardman, OH-PA",
      "29460": "Lakeland-Winter Haven, FL",
      "14540": "Bowling Green, KY",
      "49180": "Winston-Salem, NC",
      "49740": "Yuma, AZ",
      "48060": "Watertown-Fort Drum, NY",
      "29540": "Lancaster, PA",
      "39340": "Provo-Orem, UT",
      "42540": "Scranton--Wilkes-Barre--Hazelton, PA",
      "47380": "Waco, TX",
      "47260": "Virginia Beach-Norfolk-Newport News, VA-NC",
      "42060": "Santa Barbara-Santa Maria-Goleta, CA",
      "22900": "Fort Smith, AR-OK",
      "16580": "Champaign-Urbana, IL",
      "44100": "Springfield, IL",
      "25260": "Hanford-Corcoran, CA",
      "30980": "Longview, TX",
      "11540": "Appleton, WI",
      "41620": "Salt Lake City, UT",
      "41740": "San Diego-Carlsbad, CA",
      "10580": "Albany-Schenectady-Troy, NY",
      "35980": "Norwich-New London, CT",
      "10900": "Allentown-Bethlehem-Easton, PA-NJ",
      "35620": "New York-Newark- Jersey City, NY-NJ-PA (White Plains central city recoded to balance of metropolitan)",
      "15680": "California-Lexington Park, MD",
      "42140": "Santa Fe, NM",
      "37100": "Oxnard-Thousand Oaks-Ventura, CA",
      "22660": "Fort Collins, CO",
      "22420": "Flint, MI",
      "19100": "Dallas-Fort Worth-Arlington, TX",
      "29820": "Las Vegas-Henderson-Paradise, NV",
      "39740": "Reading, PA",
      "18140": "Columbus, OH",
      "36100": "Ocala, FL",
      "36220": "Odessa, TX",
      "12020": "Athens-Clarke County, GA",
      "22500": "Florence, SC",
      "26420": "Houston-Baytown-Sugar Land, TX",
      "21500": "Erie, PA",
      "40140": "Riverside-San Bernardino-Ontario, CA",
      "30460": "Lexington-Fayette, KY",
      "25540": "Hartford-West Hartford-East Hartford, CT",
      "10180": "Abilene, TX",
      "28700": "Kingsport-Bristol, TN-VA",
      "28020": "Kalamazoo-Portage, MI",
      "48620": "Wichita, KS",
      "30780": "Little Rock-North Little Rock, AR",
      "41420": "Salem, OR",
      "45460": "Terre Haute, IN",
      "43620": "Sioux Falls, SD",
      "37860": "Pensacola-Ferry Pass-Brent, FL",
      "35840": "North-Port-Sarasota-Bradenton, FL",
      "46540": "Utica-Rome, NY",
      "13740": "Billings, MT",
      "22220": "Fayetteville-Springdale-Rogers, AR-MO",
      "21340": "El Paso, TX",
      "23060": "Fort Wayne, IN",
      "47940": "Waterloo-Cedar Falls, IA",
      "26900": "Indianapolis-Carmel-Anderson, IN",
      "20100": "Dover, DE",
      "10420": "Akron, OH",
      "42220": "Santa Rosa, CA",
      "32780": "Medford, OR",
      "16980": "Chicago-Naperville-Elgin, IL-IN-WI",
      "47220": "Vineland-Bridgeton, NJ",
      "44180": "Springfield, MO",
      "44060": "Spokane-Spokane Valley, WA",
      "31420": "Macon, GA",
      "31540": "Madison, WI",
      "36540": "Omaha-Council Bluffs, NE-IA",
      "31180": "Lubbock, TX",
      "24780": "Greenville, NC",
      "35300": "New Haven-Milford, CT",
      "23580": "Gainesville, GA",
      "12220": "Auburn-Opelika, AL",
      "12100": "Atlantic City-Hammonton, NJ",
      "17660": "Coeur d'Alene, ID",
      "14010": "Bloomington, IL",
      "29180": "Lafayette, LA",
      "45300": "Tampa-St. Petersburg-Clearwater, FL",
      "36260": "Ogden-Clearfield, UT",
      "27780": "Johnstown, PA",
      "33780": "Monroe, MI",
      "29700": "Laredo, TX",
      "41940": "San Jose-Sunnyvale-Santa Clara, CA",
      "35380": "New Orleans-Metairie, LA",
      "17140": "Cincinnati, OH-KY-IN",
      "27740": "Johnson City, TN",
      "46140": "Tulsa, OK",
      "33740": "Monroe, LA",
      "47580": "Warner Robins, GA",
      "12940": "Baton Rouge, LA",
      "19660": "Deltona-Daytona Beach-Ormond Beach, FL",
      "19300": "Daphne-Fairhope-Foley, AL",
      "29340": "Lake Charles, LA",
      "48660": "Wichita Falls, TX",
      "42660": "Seattle-Tacoma-Bellevue, WA",
      "38940": "Port St. Lucie-Fort Pierce, FL",
      "25180": "Hagerstown-Martinsburg, MD-WV",
      "41540": "Salisbury, MD-DE",
      "14500": "Boulder, CO",
      "27140": "Jackson, MS",
      "46340": "Tyler, TX",
      "12580": "Baltimore-Columbia-Towson, MD",
      "32820": "Memphis, TN-MS-AR",
      "40420": "Rockford, IL",
      "31140": "Louisville/Jefferson, KY-IN",
      "28420": "Kennewick-Richland, WA",
      "49340": "Worcester, MA-CT",
      "42020": "San Luis Obispo-Paso Robles-Arroyo Grande, CA",
      "46060": "Tucson, AZ",
      "43900": "Spartanburg, SC",
      "22520": "Florence-Muscle Shoals, AL",
      "24140": "Goldsboro, NC",
      "46700": "Vallejo-Fairfield, CA",
      "42100": "Santa Cruz-Watsonville, CA",
      "12260": "Augusta-Richmond County, GA-SC",
      "33860": "Montgomery, AL",
      "34820": "Myrtle Beach-Conway-North Myrtle Beach, SC-NC",
      "40380": "Rochester, NY",
      "39140": "Prescott, AZ",
      "24340": "Grand Rapids-Wyoming, MI",
      "10740": "Albuquerque, NM",
      "28140": "Kansas City, MO-KS",
      "40220": "Roanoke, VA",
      "20500": "Durham-Chapel Hill, NC",
      "14860": "Bridgeport-Stamford-Norwalk, CT",
      "45220": "Tallahassee, FL",
      "49020": "Winchester, VA-WV",
      "17900": "Columbia, SC",
      "19740": "Denver-Aurora-Lakewood, CO",
      "12620": "Bangor, ME",
      "23540": "Gainesville, FL",
      "36740": "Orlando-Kissimmee-Sanford, FL",
      "48700": "Williamsport, PA",
      "46520": "Urban Honolulu, HI",
      "19380": "Dayton, OH",
      "41500": "Salinas, CA",
      "20700": "East Stroudsburg, PA",
      "28660": "Killeen-Temple-Fort Hood, TX",
      "12060": "Atlanta-Sandy Springs-Roswell, GA",
      "15540": "Burlington-South Burlington, VT",
      "40980": "Saginaw, MI",
      "34980": "Nashville-Davidson-Murfreesboro, TN",
      "45060": "Syracuse, NY",
      "42340": "Savannah, GA",
      "27500": "Janesville-Beloit, WI",
      "17820": "Colorado Springs, CO",
      "47900": "Washington-Arlington-Alexandria, DC-VA-MD-WV",
      "16620": "Charleston, WV",
      "16060": "Carbondale-Marion, IL",
      "22180": "Fayetteville, NC",
      "15980": "Cape Coral-Fort Myers, FL",
      "25420": "Harrisburg-Carlisle, PA",
      "30340": "Lewiston-Auburn, ME",
      "15500": "Burlington, NC",
      "11700": "Asheville, NC",
      "45820": "Topeka, KS",
      "45940": "Trenton, NJ",
      "18580": "Corpus Christi, TX",
      "24580": "Green Bay, WI",
      "17460": "Cleveland-Elyria, OH",
      "15380": "Buffalo-Cheektowaga-Niagara Falls, NY",
      "34580": "Mount Vernon-Anacortes, WA",
      "44140": "Springfield, MA",
      "35660": "Niles-Benton Harbor, MI",
      "26620": "Huntsville, AL",
      "33700": "Modesto, CA",
      "26980": "Iowa City, IA",
      "16820": "Charlottesville, VA",
      "12980": "Battle Creek, MI",
      "37900": "Peoria, IL",
      "17300": "Clarksville, TN-KY",
      "13980": "Blacksburg-Christiansburg-Radford, VA",
      "23420": "Fresno, CA",
      "27340": "Jacksonville, NC",
      "33100": "Miami-Fort Lauderdale-West Palm Beach, FL",
      "25940": "Hilton Head Island-Bluffton-Beaufort, SC",
      "38900": "Portland-Vancouver-Hillsboro, OR-WA",
      "41860": "San Francisco-Oakland-Hayward, CA",
      "45780": "Toledo, OH",
      "13140": "Beaumont-Port Arthur, TX",
      "43340": "Shreveport-Bossier City, LA",
      "16540": "Chambersburg-Waynesboro, PA",
      "40060": "Richmond, VA",
      "41700": "San Antonio-New Braunfels, TX",
      "17020": "Chico, CA",
      "27980": "Kahului-Wailuku-Lahaina, HI",
      "43300": "Sherman-Dennison, TX",
      "43780": "South Bend-Mishawaka, IN-MI",
      "12700": "Barnstable Town, MA",
      "15180": "Brownsville-Harlingen, TX",
      "19780": "Des Moines-West Des Moines, IA",
      "34940": "Naples-Immokalee-Marco Island, FL",
      "41100": "St. George, UT",
      "36780": "Oshkosh-Neenah, WI",
      "11100": "Amarillo, TX",
      "39580": "Raleigh, NC",
      "24540": "Greeley, CO",
      "29200": "Lafayette-West Lafayette, IN",
      "47300": "Visalia-Porterville, CA",
      "26580": "Huntington-Ashland, WV-KY-OH",
      "17780": "College Station-Bryan, TX",
      "38220": "Pine Bluff, AR",
      "29620": "Lansing-East Lansing, MI",
      "48140": "Wausau, WI",
      "14020": "Bloomington, IN",
      "39540": "Racine, WI",
      "24860": "Greenville-Anderson-Mauldin, SC",
      "16300": "Cedar Rapids, IA",
      "21660": "Eugene, OR",
      "38300": "Pittsburgh, PA",
      "21780": "Evansville, IN-KY",
      "34060": "Morgantown, WV",
      "34740": "Muskegon, MI",
      "14460": "Boston-Cambridge-Newton, MA-NH",
      "13460": "Bend-Redmond, OR",
      "16740": "Charlotte-Concord-Gastonia, NC-SC",
      "16860": "Chattanooga, TN-GA",
      "38860": "Portland-South Portland, ME",
      "38060": "Phoenix-Mesa-Scottsdale, AZ",
      "28940": "Knoxville, TN",
      "49620": "York-Hanover, PA",
      "39820": "Redding, CA",
      "13780": "Binghamton, NY",
      "40900": "Sacramento--Arden-Arcade-Roseville, CA",
      "25860": "Hickory-Morganton-Lenoir, NC",
      "21140": "Elkhart-Goshen, IN",
      "15940": "Canton-Massillon, OH",
      "24660": "Greensboro-High Point, NC",
      "36420": "Oklahoma City, OK",
      "27260": "Jacksonville, FL",
      "37980": "Philadelphia-Camden-Wilmington, PA-NJ-DE",
      "31080": "Los Angeles-Long Beach-Anaheim, CA (Note the CBSA code change between 2003 and 2013)",
      "22140": "Farmington, NM",
      "22020": "Fargo, ND-MN",
      "16700": "Charleston-North Charleston, SC",
      "44700": "Stockton-Lodi, CA",
      "11460": "Ann Arbor, MI"
    }

# Generating city names based on GTCBSA using GTCBSA_dict
df['City'] = df['GTCBSA'].map(GTCBSA_dict)

# Dropping NaN values to remove uncategorized/non-city (GTCBSA == 0) entries
df = df.dropna(subset=['City'])

# Export to drive
df.to_csv('df.csv')
files.download('df.csv')





<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Year,GTCBSA,PEHRUSLT,PEDIPGED,PECYC,PEAFEVER,PEAFNOW,Hours Usually Worked,PEERNLAB,Household Income,Household Members,HS or GED,Years of College,Ever in Military,Currently in Military,Union Member,City
1,23,10180,19.890411,-0.589041,-0.315068,1.000000,1.301370,0.547945,-0.671233,11.958904,3.794521,0.191781,0.356164,0.080000,0.0,0.041096,"Abilene, TX"
2,23,10420,20.547771,-0.605096,-0.070064,1.420382,1.560510,-0.210191,-0.745223,13.388535,2.808917,0.197452,0.445860,0.076923,0.0,0.012739,"Akron, OH"
3,23,10580,16.427711,-0.572289,-0.253012,1.379518,1.530120,0.445783,-0.692771,12.819277,3.048193,0.204819,0.349398,0.095588,0.0,0.036145,"Albany-Schenectady-Troy, NY"
4,23,10740,16.102881,-0.544582,-0.105624,1.411523,1.543210,0.285322,-0.681756,12.323731,3.000000,0.216735,0.447188,0.079734,0.0,0.010974,"Albuquerque, NM"
5,23,10900,18.598684,-0.592105,-0.276316,1.256579,1.348684,0.585526,-0.697368,12.730263,3.210526,0.203947,0.368421,0.043103,0.0,0.013158,"Allentown-Bethlehem-Easton, PA-NJ"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,23,49180,15.273438,-0.437500,-0.132812,1.234375,1.367188,1.765625,-0.648438,10.617188,3.078125,0.257812,0.398438,0.081633,0.0,0.000000,"Winston-Salem, NC"
257,23,49340,18.201258,-0.383648,-0.163522,1.468553,1.594340,0.317610,-0.606918,12.811321,3.289308,0.292453,0.389937,0.059925,0.0,0.031447,"Worcester, MA-CT"
258,23,49620,17.539683,-0.253968,-0.380952,1.428571,1.523810,-0.349206,-0.634921,13.746032,3.317460,0.365079,0.269841,0.057692,0.0,0.015873,"York-Hanover, PA"
259,23,49660,14.151079,-0.244604,-0.309353,1.294964,1.417266,0.942446,-0.755396,11.230216,3.143885,0.352518,0.345324,0.100000,0.0,0.014388,"Youngstown-Warren-Boardman, OH-PA"
